# Notebook de Estrutura de Dados e Algoritmos de Ordenação

Centro de Ensino Unificado de Brasília - UniCeub

Professor: Pedro Henrique Rodrigues Mendes

2° Semestre de 2022

**Curso:** Ciência de Dados e Machinelearning - 2º semestre

**Aluno:** Guilherme Dias Barbosa - 22209454

## Introdução

Área onde o aluno deverá detalhar e introdudizr o leitor ao problema: [Compatilhamento de Bicicletas em Brasília](https://www.kaggle.com/datasets/joseguilhermelopes/bike-sharing-system-in-brasilia-brazil). 

A abordagem dever ser feita nos seguintes pontos:
- **História do serviço de bicicletas compartilhadas em Brasilia;**
- **Gloassário de dados (Descrever os campos dos datasets).**


## Importação dos Dados

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
df_rides = pd.read_csv('df_rides.csv', sep=',')
df_stations = pd.read_csv('df_stations.csv', sep=',')

## Formatação dos Dados

In [5]:
# transformando a coluna user_birthdate no tipo data
df_rides['user_birthdate'] = df_rides['user_birthdate'].astype("datetime64")

## Listas e Filas

O Aluno deverá implementar um sistema de **Fila (first-in first-out) e Pilha (last in, first out)** com base na classe "rides" abaixo. 

O objetivo é criar um "buffer de requisições" utilizando a Classe Ride, portanto, o usuário quando solicita um serviço de bicicleta, a requisição deve organizada em uma Fila (first-in first-out) ou Pilha (last in, first out) para ser processada no sistema.

**A classe Ride pode ser alterada para adequação ao sistemas de fila e pilha, sempre detalhando as mudanças no código.**

**Sempre comentando código e detalhando o embasamento teórico para implementação da técnica.** 

In [6]:
class ride:
    
    uf_array = ['AP', 'DF', 'SP', 'RS', 'PR', 'RJ', 'GO', 
                'AC', 'MA', 'PA', 'MG', 'RN', 'PE', 'MT', 
                'ES', 'PB', 'RR', 'BA', 'AL', 'PI', 'CE', 
                'SC', 'RO', 'AM', 'MS', 'TO', 'SE'] 
    
    gender_array = ['M','F']
    
    def __init__(self, gender, birthdate, residence, station_start, proximo = None):
        
        self.db = df_rides
        self.db_stations = df_stations
        self.proximo = proximo
        
        status = True
        
        if gender in ride.gender_array:
            pass
        else:
            status = False
            print("Não Possui essa informação de genero na base da dados")
            
        if  station_start in self.db_stations.station.unique():
            pass
        else:
            print("Não Possui essa informação de estação na base da dados")
            status = False    
            
        if  residence in ride.uf_array:
            pass
        else:
            print("Não Possui essa informação de UF brasileiro na base da dados")
            status = False 
        
        if status:
            self.status = "Iniciado"
            self.gender = gender
            self.birthdate = birthdate
            self.residence = residence
            self.ride_date =  pd.Timestamp.now().strftime('%Y-%m-%d')
            self.time_start = pd.Timestamp.now().strftime('%H:%M:%S')
            self.station_start = station_start
            
            
            self.index = len(self.db.index)
            
            self.time_end = None
            self.station_end = None
            self.ride_duration = None
            self.ride_late = None
            
            insert_list = [self.gender,
                          self.birthdate,
                          self.residence,
                          self.ride_date,
                          self.time_start,
                          self.time_end,
                          self.station_start,
                          self.station_end,
                          self.ride_duration,
                          self.ride_late]
            
            print(insert_list)
            self.db.loc[self.index] = insert_list
            
        else:
            print('Erro de Validação ou insert na base')
                
    
    def user_age(self):
        print(int(str((pd.Timestamp.now() - pd.to_datetime(self.birthdate, format='%Y-%m-%d') ) /365)[:2].strip()))
        
    def get(self):
        
        get_list = [self.gender,
                    self.birthdate,
                    self.residence,
                    self.ride_date,
                    self.time_start,
                    self.time_end,
                    self.station_start,
                    self.station_end,
                    self.ride_duration,
                    self.ride_late]
        
        print(get_list)
        
    def finish_ride(self, station_end):
        
        status =  True
        
        if  station_end in self.db_stations.station.unique():
            pass
        else:
            print("Não Possui essa informação como padrão na base da dados")
            status = False
        
        if status:
            
            self.status = "Concluido"
            self.time_end = pd.Timestamp.now().strftime('%H:%M:%S')
            self.station_end =  station_end
            self.ride_duration = pd.Timedelta(pd.Timestamp.now() - pd.to_datetime(self.ride_date + ' ' + self.time_end)) / pd.Timedelta('60s')
            
            if self.ride_duration >= 60:
                self.ride_late =   1
            else:
                self.ride_late =   0
            
            insert_list = [self.gender,
                          self.birthdate,
                          self.residence,
                          self.ride_date,
                          self.time_start,
                          self.time_end,
                          self.station_start,
                          self.station_end,
                          "{0:0.2f}".format(self.ride_duration),
                          self.ride_late]
            
            print(insert_list)
            self.db.loc[self.index] = insert_list
            
        else:
            print('Erro de Validação ou insert na base')            
            

### Criando um nó de cada tipo

In [7]:
class Node_fila:
    def __init__(self,dado = None, proximo = None):
        self.dado = dado    
        self.proximo = proximo
        
class Node_pilha:
    def __init__(self,dado = None, anterior = None):
        self.dado = dado    
        self.anterior = anterior     

### Imports Relevantes

In [8]:

import random
from pprint import pprint
from re import L
import time
from ctypes import pointer


### Fila (first-in first-out)

In [9]:
class ListaLincada_fila:
    def __init__(self):
        self.cabeca = None
        self.tamanho = 0
    
    def getno(self, index):
        pointer = self.cabeca
        for i in range(index):
            if pointer:
                pointer = pointer.proximo
            else:
                raise IndexError("Index fora de alcance")
        return pointer
        
    
    def adiciona_fila(self, elemento):
        if self.cabeca:
            pointer = self.cabeca
            while(pointer.proximo):
                pointer = pointer.proximo
            pointer.proximo = Node_fila(elemento)
        else:
            self.cabeca = Node_fila(elemento)
        self.tamanho += 1
        
    def remove_fila(self):
            self.cabeca = self.cabeca.proximo
            self.tamanho -= 1
        
    def __len__(self):
        return self.tamanho   

lista = ListaLincada_fila()

def gerar_ride_fila():
    ride_gerada = ride(random.choice(ride.gender_array),
        str(df_rides.user_birthdate[random.randint(0,287321)])[0:10],
        random.choice(ride.uf_array),
        str(df_rides.station_start[random.randint(0,287320)]).strip()
        )
    lista.adiciona_fila(ride_gerada)
    
for x in range(6):
    gerar_ride_fila()    

print("---------")


def finalizar_ride_fila():
    lista.getno(0).dado.finish_ride(df_stations.station[random.randint(0,48)])
    lista.remove_fila()
    
for x in range(lista.tamanho):    
    finalizar_ride_fila()
    time.sleep(1)

['M', '1978-08-25', 'AP', '2022-09-29', '17:48:00', None, '35 - CLN 403', None, None, None]
['M', '1990-11-07', 'AC', '2022-09-29', '17:48:01', None, '31 - EQS 204/205', None, None, None]
['M', '1996-01-23', 'AC', '2022-09-29', '17:48:01', None, '31 - EQS 204/205', None, None, None]
['F', '1980-06-21', 'AP', '2022-09-29', '17:48:01', None, '16 - SRTVS', None, None, None]
['F', '1969-11-23', 'DF', '2022-09-29', '17:48:01', None, '10 - Ministério dos Transportes', None, None, None]
['M', '1993-10-26', 'MS', '2022-09-29', '17:48:01', None, '5 - Setor Hoteleiro Norte', None, None, None]
---------
['M', '1978-08-25', 'AP', '2022-09-29', '17:48:00', '17:48:01', '35 - CLN 403', '20 - SAS', '0.01', 0]
['M', '1990-11-07', 'AC', '2022-09-29', '17:48:01', '17:48:02', '31 - EQS 204/205', '33 - EQS 104/304', '0.01', 0]
['M', '1996-01-23', 'AC', '2022-09-29', '17:48:01', '17:48:03', '31 - EQS 204/205', '38 - SQN 205', '0.01', 0]
['F', '1980-06-21', 'AP', '2022-09-29', '17:48:01', '17:48:04', '16 - S

### Pilha (last in, first out)

In [10]:
class ListaLincada_pilha:
    def __init__(self):
        self.topo = None
        self.tamanho = 0
 
    def getno(self, index):
        pointer = self.topo
        for i in range(index):
            if pointer:
                pointer = pointer.anterior
            else:
                raise IndexError("Index fora de alcance")
        
        return pointer
        
    def adiciona_pilha(self, elemento):
        pointer = Node_pilha(elemento)
        pointer.anterior = self.topo
        self.topo = pointer
        self.tamanho += 1         
        
    def remove_pilha(self):
        self.topo = self.topo.anterior
        self.tamanho -= 1
        
    def __len__(self):
        return self.tamanho   

#############################################################
    
lista_pilha = ListaLincada_pilha()
    
def gerar_ride_pilha():
    ride_gerada = ride(random.choice(ride.gender_array),
        str(df_rides.user_birthdate[random.randint(0,287321)])[0:10],
        random.choice(ride.uf_array),
        str(df_rides.station_start[random.randint(0,287320)]).strip()
        )
    lista_pilha.adiciona_pilha(ride_gerada)
    
for x in range(6):
    gerar_ride_pilha()    

print("---------")

def finalizar_ride_pilha():
    lista_pilha.getno(-1).dado.finish_ride(df_stations.station[random.randint(0,48)])
    lista_pilha.remove_pilha()

for x in range(lista_pilha.tamanho):
    finalizar_ride_pilha()


['F', '1996-01-31', 'PR', '2022-09-29', '17:48:07', None, '35 - CLN 403', None, None, None]
['M', '1998-02-06', 'AP', '2022-09-29', '17:48:07', None, '16 - SRTVS', None, None, None]
['M', '1974-10-06', 'GO', '2022-09-29', '17:48:07', None, '37 - EQN 104/105', None, None, None]
['F', '1972-05-09', 'MA', '2022-09-29', '17:48:07', None, '41 - Instituto de Artes', None, None, None]
['M', '1995-01-20', 'SP', '2022-09-29', '17:48:07', None, '45 - Centro Olímpico', None, None, None]
['F', '2020-03-25', 'RN', '2022-09-29', '17:48:07', None, '18 - Praça do Povo', None, None, None]
---------
['F', '2020-03-25', 'RN', '2022-09-29', '17:48:07', '17:48:07', '18 - Praça do Povo', '8 - Ministério da Defesa', '0.01', 0]
['M', '1995-01-20', 'SP', '2022-09-29', '17:48:07', '17:48:07', '45 - Centro Olímpico', '500 - Deck Sul Kids', '0.01', 0]
['F', '1972-05-09', 'MA', '2022-09-29', '17:48:07', '17:48:07', '41 - Instituto de Artes', '3 - Centro de Convenções', '0.01', 0]
['M', '1974-10-06', 'GO', '2022-09

## Algortimos de Ordenação
O Aluno desenvolve os algoritmos de ordenação abaixo, com o objetivo de ordenar o dataset **df_stations** embaralhado.

**Realizar a Ánalise de Complexidade nos algoritmos (Notação Big O e Melhor Caso/Pior Caso)**

**Sempre comentando código e detalhando o embasamento teórico para implementação da técnica.** 

In [11]:
df_stations.head(5)

,station,station_number,station_name,lat,lon
0,1 - Memorial JK,1,Memorial JK,-15.783479,-47.913372
1,2 - Praça Buriti,2,Praça Buriti,-15.785651,-47.908750
2,3 - Centro de Convenções,3,Centro de Convenções,-15.786409,-47.899865
3,4 - Torre de TV,4,Torre de TV,-15.789581,-47.894312
4,5 - Setor Hoteleiro Norte,5,Setor Hoteleiro Norte,-15.789029,-47.891015


In [71]:
df_stations = df_stations.sample(frac=1,random_state=42)

In [72]:
df_stations.head(5)

,station,station_number,station_name,lat,lon
8,9 - Ministério da Cultura,9,Ministério da Cultura,-15.798295,-47.871807
42,43 - Biblioteca Central,43,Biblioteca Central,-15.761123,-47.867443
28,29 - INMETRO,29,INMETRO,-15.780858,-47.883277
7,8 - Ministério da Defesa,8,Ministério da Defesa,-15.795867,-47.870901
41,42 - PAT,42,PAT,-15.759220,-47.870397


### Bubblesort

In [67]:
def bubblesort(df):
    array = [i for i in df.index]
    for i in range(len(array) - 1):
        for j in range(len(array) - 1 - i):
            if array[j] > array[j + 1]:
                temp = array[j]
                array[j] = array[j + 1]
                array[j + 1] = temp
    return df.reindex(array)

df_stations = bubblesort(df_stations)

### Mergesort

In [ ]:
def mergesort(df):
    array = [i for i in df.index]
    print("Splitting ", array)
    if len(array) > 1:
        mid = len(array) // 2
        lefthalf = array[:mid]
        righthalf = array[mid:]

        mergesort(lefthalf)
        mergesort(righthalf)

        i = 0
        j = 0
        k = 0
        while i < len(lefthalf) and j < len(righthalf):
            if lefthalf[i] < righthalf[j]:
                array[k] = lefthalf[i]
                i = i + 1
            else:
                array[k] = righthalf[j]
                j = j + 1
            k = k + 1

        while i < len(lefthalf):
            array[k] = lefthalf[i]
            i = i + 1
            k = k + 1

        while j < len(righthalf):
            array[k] = righthalf[j]
            j = j + 1
            k = k + 1
    print("Merging ", array)
    return df_stations.reindex(array)

array = [i for i in df_stations.index]
df_stations = mergesort(array)
    

### Heapsort

In [63]:
def heapify(array, n, i):
    maior = i
    l = 2 * i + 1
    r = 2 * i + 2

    if l < n and array[i] < array[l]:
        maior = l

    if r < n and array[maior] < array[r]:
        maior = r

    if maior != i:
        array[i], array[maior] = array[maior], array[i]
        heapify(array, n, maior)

def heapsort(array):
    n = len(array)

    # Construir heap máximo
    for i in range(n // 2, -1, -1):
        heapify(array, n, i)

    for i in range(n - 1, 0, -1):
        # Troca
        array[i], array[0] = array[0], array[i]

        # Elemento raiz do empilhamento
        heapify(array, i, 0)



### Quicksort